In [1]:
from mxnet import gluon
import mxnet as mx
from mxnet import nd
from mxnet.gluon import nn  

/home/jiancheng/anaconda3/lib/python3.6/site-packages/urllib3/contrib/pyopenssl.py:46: DeprecationWarning: OpenSSL.rand is deprecated - you should use os.urandom instead
  import OpenSSL.SSL


In [2]:
from tmpref.model import Deeplabv3

Using TensorFlow backend.
/home/jiancheng/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/ops.py:766: DeprecationWarning: builtin type EagerTensor has no __module__ attribute
  EagerTensor = c_api.TFE_Py_InitEagerTensor(_EagerTensorBase)
/home/jiancheng/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)
/home/jiancheng/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [3]:
import keras.backend as K
import tensorflow as tf

In [4]:
def get_gpu_session(ratio=None, interactive=False):
    config = tf.ConfigProto(allow_soft_placement=True)
    if ratio is None:
        config.gpu_options.allow_growth = True
    else:
        config.gpu_options.per_process_gpu_memory_fraction = ratio
    if interactive:
        sess = tf.InteractiveSession(config=config)
    else:
        sess = tf.Session(config=config)
    return sess


def set_gpu_usage(ratio=None):
    sess = get_gpu_session(ratio)
    K.set_session(sess)

In [5]:
set_gpu_usage()

In [6]:
keras_model = Deeplabv3(backbone="xception")

/home/jiancheng/anaconda3/lib/python3.6/site-packages/tensorflow/python/util/tf_inspect.py:45: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() or inspect.getfullargspec()
  if d.decorator_argspec is not None), _inspect.getargspec(target))


In [7]:
keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 256, 256, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [11]:
# from keras.utils import plot_model
# plot_model(keras_model,to_file='xception.jpg',show_shapes=True)

In [9]:
for layer in keras_model.layers:
    print(layer.name)

input_1
entry_flow_conv1_1
entry_flow_conv1_1_BN
activation_1
entry_flow_conv1_2
entry_flow_conv1_2_BN
activation_2
activation_3
entry_flow_block1_separable_conv1_depthwise
entry_flow_block1_separable_conv1_depthwise_BN
entry_flow_block1_separable_conv1_pointwise
entry_flow_block1_separable_conv1_pointwise_BN
activation_4
entry_flow_block1_separable_conv2_depthwise
entry_flow_block1_separable_conv2_depthwise_BN
entry_flow_block1_separable_conv2_pointwise
entry_flow_block1_separable_conv2_pointwise_BN
zero_padding2d_1
activation_5
entry_flow_block1_separable_conv3_depthwise
entry_flow_block1_separable_conv3_depthwise_BN
zero_padding2d_2
entry_flow_block1_separable_conv3_pointwise
entry_flow_block1_shortcut
entry_flow_block1_separable_conv3_pointwise_BN
entry_flow_block1_shortcut_BN
add_1
activation_6
entry_flow_block2_separable_conv1_depthwise
entry_flow_block2_separable_conv1_depthwise_BN
entry_flow_block2_separable_conv1_pointwise
entry_flow_block2_separable_conv1_pointwise_BN
activat

In [12]:
def compute_same_padding(kernel_size, dilation):
    #TODO: compute `same` padding for stride<=2 ? 
    kernel_size_effective = kernel_size + (kernel_size - 1) * (dilation - 1)
    pad_total = kernel_size_effective - 1
    pad_beg = pad_total // 2
    pad_end = pad_total - pad_beg
    return pad_beg, pad_end

In [232]:
class SeparableConv(nn.HybridBlock):
    #TODO: should I fix epsilon in BN layer?
    
    def __init__(self, out_filters, kernel_size, strides, dilation, depth_activation, in_filters=None, prefix=None):
        super(SeparableConv,self).__init__(prefix=prefix)
        
        if in_filters is None:
            in_filters = out_filters
        
        self.depth_activation = depth_activation

        padding = compute_same_padding(kernel_size, dilation)
        # filter_in==filter_out
        with self.name_scope():
            self.depthwise_conv = nn.Conv2D(in_filters, in_channels=in_filters, groups=in_filters, 
                                            dilation=dilation,use_bias=False,
                                            padding=padding, strides=strides,
                                            kernel_size=kernel_size,prefix='depthwise_')
            self.bn1 = nn.BatchNorm(axis=1,prefix='depthwise_BN_')
            self.pointwise_conv = nn.Conv2D(out_filters, kernel_size=1,use_bias=False, prefix='pointwise_')
            self.bn2 = nn.BatchNorm(axis=1,prefix='pointwise_BN_')
        
    def hybrid_forward(self, F, x):
        if not self.depth_activation:
            x = F.relu(x)
        x = self.depthwise_conv(x)
        x = self.bn1(x)
        if self.depth_activation:
            x = F.relu(x)
        x = self.pointwise_conv(x)
        x = self.bn2(x)
        if self.depth_activation:
            x = F.relu(x)
        return x   


In [233]:
# ?nn.BatchNorm(axis=1)

In [234]:
inputs = nd.random_normal(shape=(8,16,48,48),ctx=mx.gpu(0))

In [235]:
sep_conv = SeparableConv(16,3,1,1,False)

In [236]:
sep_conv.initialize(ctx=mx.gpu(0))

In [237]:
sep_conv(inputs).shape

(8, 16, 48, 48)

In [238]:
# ?nn.Conv2D

In [239]:
sep_conv.depthwise_conv.name

'separableconv3_depthwise'

In [281]:
class XceptionBlock(nn.HybridBlock):
    
    def __init__(self, filters_list, kernel_size, strides, dilation, depth_activation, in_filters=None, prefix=None):
        super(XceptionBlock,self).__init__(prefix=prefix)
        
        assert len(filters_list)==3
        
        if in_filters is None:
            in_filters = filters_list[0]
        
        with self.name_scope():
            self.conv1 = SeparableConv(filters_list[0], kernel_size, 1, dilation, depth_activation,
                                       in_filters=in_filters, prefix='separable_conv1_')
            self.conv2 = SeparableConv(filters_list[1], kernel_size, 1, dilation, depth_activation,
                                       in_filters=filters_list[1],prefix='separable_conv2_')
            self.conv3 = SeparableConv(filters_list[2], kernel_size, strides, dilation, depth_activation,
                                       in_filters=filters_list[2],prefix='separable_conv3_')
            if strides>1:
                self.shortcut_conv = nn.Conv2D(filters_list[-1], kernel_size=1, strides=strides,use_bias=False,prefix='shortcut_')
                self.shortcut_bn = nn.BatchNorm(axis=1,prefix='shortcut_BN_')
        
        self.strides = strides
        
    def hybrid_forward(self, F, x, return_skip=False):
        shortcut = x
        x = self.conv1(x)
        x = self.conv2(x)
        skip = x
        x = self.conv3(x)
        if self.strides>1:
            shortcut = self.shortcut_conv(shortcut)
            shortcut = self.shortcut_bn(shortcut)
        if return_skip:
            return x+shortcut, skip
        return x+shortcut

In [282]:
xception_block = XceptionBlock([128,128,128],kernel_size=3, strides=1, dilation=2, depth_activation=False)

In [283]:
inputs = nd.random_normal(shape=(8,128,32,32),ctx=mx.gpu(0))

In [284]:
xception_block.initialize(ctx=mx.gpu(0))

In [285]:
xception_block(inputs)


[[[[  5.44731840e-02   2.50033021e-01  -1.22682905e+00 ...,
      5.79346061e-01  -1.37952912e+00  -7.35129356e-01]
   [ -8.47813189e-01   1.49603891e+00  -5.54334879e-01 ...,
     -4.68602553e-02   2.99147427e-01   1.56427756e-01]
   [ -1.66740692e+00   8.48381937e-01   3.44440758e-01 ...,
      1.77532479e-01  -2.51425743e-01   5.23265660e-01]
   ..., 
   [  4.08208281e-01  -9.90332544e-01  -1.57819963e+00 ...,
      1.18875444e+00  -1.94043243e+00   1.01852512e+00]
   [ -1.83000743e+00   6.05638385e-01  -1.32430047e-01 ...,
      2.57126596e-02  -4.00534838e-01  -5.41451693e-01]
   [ -4.71729010e-01  -6.17879331e-01   1.77953079e-01 ...,
     -4.37577248e-01   6.30368471e-01   8.62508953e-01]]

  [[  6.29671276e-01  -1.48638320e+00   6.85556054e-01 ...,
     -2.49717727e-01   7.46461987e-01  -1.22416580e+00]
   [  7.38662720e-01   3.73257369e-01   5.64668298e-01 ...,
     -8.07732701e-01  -4.60245550e-01   8.66749763e-01]
   [ -8.53779614e-01  -5.03252000e-02   1.58478248e+00 ...,


In [286]:
inputs = nd.random_normal(shape=(8,3,512,512),ctx=mx.gpu(0))

In [287]:
conv = nn.Conv2D(32,kernel_size=3, strides=4, padding=1)
conv.initialize(ctx=mx.gpu(0))

In [288]:
conv(inputs).shape

(8, 32, 128, 128)

In [289]:
nn.Activation("relu")

Activation(relu)

In [290]:
keras_entry_flow.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 256, 256, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [291]:
class EntryFlow(nn.HybridBlock):
    
    def __init__(self, prefix='entry_flow_'):
        super(EntryFlow,self).__init__(prefix)

        with self.name_scope():
            self.conv1 = nn.HybridSequential(prefix='conv1_')
            with self.conv1.name_scope():
                self.conv1.add(nn.Conv2D(32, kernel_size=3, strides=2, padding=1,use_bias=False,prefix='1_'))
                self.conv1.add(nn.BatchNorm(axis=1,prefix='1_BN_'))
                self.conv1.add(nn.Activation("relu"))
            self.conv2 = nn.HybridSequential(prefix='conv1_')
            with self.conv2.name_scope():
                self.conv2.add(nn.Conv2D(64, kernel_size=3, padding=1,use_bias=False,prefix='2_'))
                self.conv2.add(nn.BatchNorm(axis=1,prefix='2_BN_'))
                self.conv2.add(nn.Activation("relu"))

            self.conv3 = XceptionBlock(filters_list=[128,128,128], kernel_size=3, strides=2,
                                       dilation=1, depth_activation=False,in_filters=64,prefix='block1_')
            self.conv4 = XceptionBlock(filters_list=[256,256,256], kernel_size=3, strides=2,
                                       dilation=1, depth_activation=False, in_filters=128,prefix='block2_')
            self.conv5 = XceptionBlock(filters_list=[728,728,728], kernel_size=3, strides=2,
                                       dilation=1, depth_activation=False, in_filters=256,prefix='block3_')
        
    def hybrid_forward(self, F, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x, skip = self.conv4(x, True)
        x = self.conv5(x)
        return x, skip

In [292]:
# class 

In [293]:
entry_flow = EntryFlow()

In [294]:
entry_flow.conv3.collect_params()

entry_flow_block1_ (
  Parameter entry_flow_block1_separable_conv1_depthwise_weight (shape=(64, 1, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_BN_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_BN_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_BN_running_mean (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_BN_running_var (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_pointwise_weight (shape=(128, 0, 1, 1), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_pointwise_BN_gamma (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_pointwise_BN_beta (shape=(0,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_pointwise_BN_running_mean (shape=(0

In [296]:
len(weights)

732

In [315]:
entry_flow.collect_params()

entry_flow_ (
  Parameter entry_flow_conv1_1_weight (shape=(32, 3, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_gamma (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_beta (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_running_mean (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_1_BN_running_var (shape=(32,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_weight (shape=(64, 32, 3, 3), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_gamma (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_beta (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_running_mean (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_conv1_2_BN_running_var (shape=(64,), dtype=<class 'numpy.float32'>)
  Parameter entry_flow_block1_separable_conv1_depthwise_weight (shape=(64, 1, 3, 3), dtype=<class '

In [311]:
keras_model.weights

[<tf.Variable 'entry_flow_conv1_1/kernel:0' shape=(3, 3, 3, 32) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_1_BN/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_1_BN/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_2/kernel:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_2_BN/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_2_BN/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_depthwise/depthwise_kernel:0' shape=(3, 3, 64, 1) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_depthwise_BN/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_depthwise_BN/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_pointwise/kernel:0' shape=(1, 1, 64, 128) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_pointwise_BN/gamma:0' shape=(128,) dt

In [318]:
x = np.random.rand(3,4,5)

In [319]:
x.transpose(2,0,1)

array([[[ 0.32836361,  0.86284354,  0.13419234,  0.59521181],
        [ 0.57674053,  0.08361336,  0.42088566,  0.89516569],
        [ 0.77262125,  0.31647994,  0.26075849,  0.03986466]],

       [[ 0.18081635,  0.73991884,  0.04905462,  0.83926737],
        [ 0.13615929,  0.92477592,  0.50640254,  0.57243992],
        [ 0.10922309,  0.02832224,  0.72674389,  0.35769726]],

       [[ 0.43383194,  0.95743706,  0.27392486,  0.01107989],
        [ 0.77978516,  0.27744802,  0.55242688,  0.63491381],
        [ 0.38689212,  0.89116808,  0.70028746,  0.0719273 ]],

       [[ 0.30959343,  0.55783304,  0.33826844,  0.4208095 ],
        [ 0.26222309,  0.20505461,  0.20335557,  0.07620751],
        [ 0.51424082,  0.06531187,  0.92883262,  0.94508968]],

       [[ 0.82173491,  0.08882553,  0.19676402,  0.01269014],
        [ 0.78325615,  0.72692678,  0.61742688,  0.40561938],
        [ 0.3364681 ,  0.5736849 ,  0.3993309 ,  0.92563529]]])

In [350]:
class WeightConverter:
    
    def __init__(self, keras_model=None, weight_dict=None):
        if keras_model is None:
            self.weight_dict = weight_dict
        elif weight_dict is None:
            self.weight_dict = dict()
            for weight_var in tqdm(keras_model.weights):
                self.weight_dict[weight_var.name] = K.eval(weight_var)
        else:
            raise ValueError
            
    def replace_depthwise_weight(self, query, param):
        target = query.replace("_weight",'/depthwise_kernel:0')
        assert target in self.weight_dict, "%s->%s"%(k, target)
        weight = self.weight_dict[target]
        param.set_data(weight.transpose(3,2,0,1))
        
    def replace_weight(self, query, param):
        target = query.replace("_weight",'/kernel:0')
        assert target in self.weight_dict, "%s->%s"%(k, target)
        weight = self.weight_dict[target]
        param.set_data(weight.transpose(3,2,0,1))
        
    def replace_gamma(self, query, param):
        target = query.replace("_gamma",'/gamma:0') 
        assert target in self.weight_dict, "%s->%s"%(k, target)
        weight = self.weight_dict[target]
#         param.set_data(weight.transpose(3,2,0,1))
        
    def replace_beta(self, query, param):
        target = query.replace("_beta",'/beta:0')
        assert target in self.weight_dict, "%s->%s"%(k, target)
        weight = self.weight_dict[target]
#         param.set_data(weight.transpose(3,2,0,1))
        
    def replace_running_mean(self, query, param):
        target = query.replace("_running_mean",'/moving_mean:0')
        assert target in self.weight_dict, "%s->%s"%(k, target)
        weight = self.weight_dict[target]
#         param.set_data(weight.transpose(3,2,0,1))
        
    def replace_running_var(self, query, param):
        target = query.replace("_running_var",'/moving_variance:0')
        assert target in self.weight_dict, "%s->%s"%(k, target)
        weight = self.weight_dict[target]
#         param.set_data(weight.transpose(3,2,0,1))
        
    def set_parameters(self, gluon_model):
        for k, param in gluon_model.collect_params().items():
            if k.endswith("depthwise_weight"):
                self.replace_depthwise_weight(k, param)
            elif k.endswith("weight"):
                self.replace_weight(k, param)
            elif k.endswith("gamma"):
                self.replace_gamma(k, param)
            elif k.endswith("beta"):
                self.replace_beta(k, param)
            elif k.endswith("running_mean"):
                self.replace_running_mean(k, param)
            elif k.endswith("running_var"):
                self.replace_running_var(k, param)
            else:
                raise ValueError

In [351]:
converter = WeightConverter(weight_dict=weights_dict)

In [352]:
converter.set_parameters(entry_flow)

AssertionError: Expected shape (1, 64, 3, 3) is incompatible with given shape (64, 1, 3, 3).

In [353]:
gluon_weight = entry_flow.conv1[0].weight.data()

In [330]:
keras_weight = weights_dict['entry_flow_conv1_1/kernel:0']

In [331]:
keras_weight.shape

(3, 3, 3, 32)

In [332]:
gluon_weight.shape

(32, 3, 3, 3)

In [338]:
keras_weight

array([[[[  3.09258793e-02,   5.63149825e-02,   1.43224746e-01,
           -3.49594988e-02,   3.72597352e-02,   5.10606910e-13,
            4.14342036e-13,  -2.08895624e-01,  -1.49981216e-01,
            1.09545469e-01,   6.86957613e-02,  -1.80448610e-02,
           -4.57622334e-02,  -1.94059182e-02,   3.96955945e-02,
            4.43271212e-02,   4.95776124e-02,   4.80280229e-04,
           -1.37275755e-01,  -1.80539871e-12,   1.42697711e-02,
            1.30766869e-01,  -1.39577640e-02,  -2.72624567e-09,
           -4.78190668e-02,   6.64784461e-02,  -1.64946001e-02,
           -6.30989969e-02,   7.25580787e-04,  -5.09459479e-03,
           -5.82973324e-02,  -4.72557207e-04],
         [  6.65731030e-03,   1.68099105e-02,  -1.53193504e-01,
            1.80230916e-01,  -5.82513250e-02,   1.18147397e-12,
           -1.15552994e-13,  -2.20524460e-01,   9.59412605e-02,
            5.90464137e-02,   4.22060564e-02,  -6.29967451e-02,
            3.05419043e-03,   4.91537936e-02,   1.8476914

In [349]:
(gluon_weight.asnumpy().transpose((2,3,1,0))-keras_weight).sum()

0.0

In [309]:
# for k, param in entry_flow.collect_params().items():
#     if k.endswith("depthwise_weight"):
#         target = k.replace("_weight",'/depthwise_kernel:0')
#     elif k.endswith("weight"):
#         target = k.replace("_weight",'/kernel:0')
#     elif k.endswith("gamma"):
#         target = k.replace("_gamma",'/gamma:0') 
#     elif k.endswith("beta"):
#         target = k.replace("_beta",'/beta:0')
#     elif k.endswith("running_mean"):
#         target = k.replace("_running_mean",'/moving_mean:0')
#     elif k.endswith("running_var"):
#         target = k.replace("_running_var",'/moving_variance:0')
#     else:
#         pass
#     assert target in weights_dict, "%s->%s"%(k, target)

In [130]:
weights = keras_model.get_weights()

In [ ]:
keras_model.get_layer()

In [133]:
weights[0]

array([[[[  3.09258793e-02,   5.63149825e-02,   1.43224746e-01,
           -3.49594988e-02,   3.72597352e-02,   5.10606910e-13,
            4.14342036e-13,  -2.08895624e-01,  -1.49981216e-01,
            1.09545469e-01,   6.86957613e-02,  -1.80448610e-02,
           -4.57622334e-02,  -1.94059182e-02,   3.96955945e-02,
            4.43271212e-02,   4.95776124e-02,   4.80280229e-04,
           -1.37275755e-01,  -1.80539871e-12,   1.42697711e-02,
            1.30766869e-01,  -1.39577640e-02,  -2.72624567e-09,
           -4.78190668e-02,   6.64784461e-02,  -1.64946001e-02,
           -6.30989969e-02,   7.25580787e-04,  -5.09459479e-03,
           -5.82973324e-02,  -4.72557207e-04],
         [  6.65731030e-03,   1.68099105e-02,  -1.53193504e-01,
            1.80230916e-01,  -5.82513250e-02,   1.18147397e-12,
           -1.15552994e-13,  -2.20524460e-01,   9.59412605e-02,
            5.90464137e-02,   4.22060564e-02,  -6.29967451e-02,
            3.05419043e-03,   4.91537936e-02,   1.8476914

In [135]:
kernel_weights = keras_model.weights[0]

In [138]:
kernel_weights.name

'entry_flow_conv1_1/kernel:0'

In [146]:
keras_model.weights

[<tf.Variable 'entry_flow_conv1_1/kernel:0' shape=(3, 3, 3, 32) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_1_BN/gamma:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_1_BN/beta:0' shape=(32,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_2/kernel:0' shape=(3, 3, 32, 64) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_2_BN/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_conv1_2_BN/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_depthwise/depthwise_kernel:0' shape=(3, 3, 64, 1) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_depthwise_BN/gamma:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_depthwise_BN/beta:0' shape=(64,) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_pointwise/kernel:0' shape=(1, 1, 64, 128) dtype=float32_ref>,
 <tf.Variable 'entry_flow_block1_separable_conv1_pointwise_BN/gamma:0' shape=(128,) dt

In [147]:
len(weights)

732

In [148]:
type(weights)

list

In [151]:
from tqdm import tqdm

In [152]:
weights_dict = dict()
for weights_var in tqdm(keras_model.weights):
    weights_dict[weights_var.name] = K.eval(weights_var)

100%|██████████| 732/732 [00:33<00:00, 21.58it/s]  


In [158]:
for k,v in entry_flow.collect_params().items():
    print(k, v.shape,np.prod(v.shape))

entryflow1_conv1_conv0_weight (32, 0, 3, 3) 0
entryflow1_conv1_BNgamma (0,) 0
entryflow1_conv1_BNbeta (0,) 0
entryflow1_conv1_BNrunning_mean (0,) 0
entryflow1_conv1_BNrunning_var (0,) 0
entryflow1_conv2_conv0_weight (64, 0, 3, 3) 0
entryflow1_conv2_BNgamma (0,) 0
entryflow1_conv2_BNbeta (0,) 0
entryflow1_conv2_BNrunning_mean (0,) 0
entryflow1_conv2_BNrunning_var (0,) 0
entryflow1_xceptionblock0_separableconv0_conv0_weight (64, 1, 3, 3) 576
entryflow1_xceptionblock0_separableconv0_batchnorm0_gamma (0,) 0
entryflow1_xceptionblock0_separableconv0_batchnorm0_beta (0,) 0
entryflow1_xceptionblock0_separableconv0_batchnorm0_running_mean (0,) 0
entryflow1_xceptionblock0_separableconv0_batchnorm0_running_var (0,) 0
entryflow1_xceptionblock0_separableconv0_conv1_weight (128, 0, 1, 1) 0
entryflow1_xceptionblock0_separableconv0_batchnorm1_gamma (0,) 0
entryflow1_xceptionblock0_separableconv0_batchnorm1_beta (0,) 0
entryflow1_xceptionblock0_separableconv0_batchnorm1_running_mean (0,) 0
entryflow1_x

In [153]:
len(weights_dict)

732

In [155]:
with open("model_weight.npz","wb") as f:
    np.savez_compressed(f, **weights_dict)

In [139]:
import keras.backend as K

In [141]:
K.eval(kernel_weights).shape

(3, 3, 3, 32)

In [115]:
class MiddleFlow:
    pass

In [116]:
class ExitFlow:
    pass

In [117]:
class ASPP:
    pass

In [118]:
class Encoder:
    pass

In [119]:
class Decoder:
    pass

In [120]:
class DeepLabv3p:
    pass

In [121]:
entry_flow

EntryFlow(
  (conv1): HybridSequential(
    (0): Conv2D(None -> 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=None)
    (2): Activation(relu)
  )
  (conv2): HybridSequential(
    (0): Conv2D(None -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=None)
    (2): Activation(relu)
  )
  (conv3): XceptionBlock(
    (conv1): SeparableConv(
      (depthwise_conv): Conv2D(1 -> 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64, bias=False)
      (bn1): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=False, in_channels=None)
      (pointwise_conv): Conv2D(None -> 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn2): BatchNorm(axis=1, eps=1e-05, momentum=0.9, fix_gamma=False, use_global_stats=Fal

In [122]:
import numpy as np

In [80]:
sum([np.prod(v.shape) for k,v in entry_flow.collect_params().items()])

1750672

In [70]:
keras_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 256, 256, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [74]:
for i, layer in enumerate(keras_model.layers):
    print(i, layer.name)

0 input_1
1 entry_flow_conv1_1
2 entry_flow_conv1_1_BN
3 activation_1
4 entry_flow_conv1_2
5 entry_flow_conv1_2_BN
6 activation_2
7 activation_3
8 entry_flow_block1_separable_conv1_depthwise
9 entry_flow_block1_separable_conv1_depthwise_BN
10 entry_flow_block1_separable_conv1_pointwise
11 entry_flow_block1_separable_conv1_pointwise_BN
12 activation_4
13 entry_flow_block1_separable_conv2_depthwise
14 entry_flow_block1_separable_conv2_depthwise_BN
15 entry_flow_block1_separable_conv2_pointwise
16 entry_flow_block1_separable_conv2_pointwise_BN
17 zero_padding2d_1
18 activation_5
19 entry_flow_block1_separable_conv3_depthwise
20 entry_flow_block1_separable_conv3_depthwise_BN
21 zero_padding2d_2
22 entry_flow_block1_separable_conv3_pointwise
23 entry_flow_block1_shortcut
24 entry_flow_block1_separable_conv3_pointwise_BN
25 entry_flow_block1_shortcut_BN
26 add_1
27 activation_6
28 entry_flow_block2_separable_conv1_depthwise
29 entry_flow_block2_separable_conv1_depthwise_BN
30 entry_flow_bloc

In [71]:
from keras.models import Model

In [84]:
keras_entry_flow = Model(inputs=keras_model.inputs, outputs=keras_model.get_layer("activation_12").output)

In [85]:
keras_entry_flow.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 512, 512, 3)  0                                            
__________________________________________________________________________________________________
entry_flow_conv1_1 (Conv2D)     (None, 256, 256, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
entry_flow_conv1_1_BN (BatchNor (None, 256, 256, 32) 128         entry_flow_conv1_1[0][0]         
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 256, 256, 32) 0           entry_flow_conv1_1_BN[0][0]      
__________________________________________________________________________________________________
entry_flow

In [61]:
?entry_flow.collect_params

In [60]:
entry_flow. entryflow0_hybridsequential0_conv0_weight

TypeError: 'ParameterDict' object is not callable

In [46]:
# 728/32

In [47]:
entry_flow = EntryFlow()

In [312]:
entry_flow.initialize(ctx=mx.gpu(0))

In [313]:
inputs = nd.random_normal(shape=(8,3,512,512),ctx=mx.gpu(0))

In [314]:
features, skip = entry_flow(inputs)

In [51]:
features.shape

(8, 728, 32, 32)

In [52]:
skip.shape

(8, 256, 128, 128)